# Generating a random sequence of musical notes with Markov Chains

## Import All Required Libraries

In [17]:
import pretty_midi as pm
import random
from IPython.display import Audio
import numpy as np
from scipy.io.wavfile import write

## Create an example note sequence

In [18]:
# List with al the notes in the C Major scale, serving as the pool of notes to generate music
notes = ['C', 'D', 'E', 'F', 'G', 'A', 'B']

## Define transition matrix for the notes

In [19]:
# Define the transition matrix, used to map each note of the C Major scale & the next possible note to follow it
transition_matrix = {
    'C': ['D', 'E', 'F', 'G', 'A', 'B'],
    'D': ['C', 'E', 'F', 'G', 'A', 'B'],
    'E': ['C', 'D', 'F', 'G', 'A', 'B'],
    'F': ['C', 'D', 'E', 'G', 'A', 'B'],
    'G': ['C', 'D', 'E', 'F', 'A', 'B'],
    'A': ['C', 'D', 'E', 'F', 'G', 'B'],
    'B': ['C', 'D', 'E', 'F', 'G', 'A'],
}

## Generate music function

In [20]:
def generate_music(start_note, length=20, octave=4):
    # All generated notes
    music = [start_note + str(octave)]
    # Current note
    current_note = start_note
    
    # Generate the set number of notes
    for _ in range(length):
        # The next note is chosen at random from the transition matrix, based on the current note
        next_note = random.choice(transition_matrix[current_note])
        # The selected note is turned into MIDI note number (Note name + Octave)
        note_number = next_note + str(octave)
        # Note is appended to the music list
        music.append(note_number)
        # The current note is updated
        current_note = next_note
    
    # Return the generated notes
    return music

## Generate a music sequence


In [21]:
generated_music = generate_music('C', length=20, octave=2)
print("Generated music sequence:", generated_music)

Generated music sequence: ['C2', 'G2', 'C2', 'B2', 'E2', 'F2', 'C2', 'A2', 'E2', 'D2', 'E2', 'F2', 'B2', 'G2', 'A2', 'G2', 'B2', 'G2', 'C2', 'B2', 'C2']


## Create the .wav File

In [22]:
def create_wav_file(gen_notes, file_path, sample_rate=44100):
    # Create a PrettyMIDI object
    midi_data = pm.PrettyMIDI()
    # Create an Instrument instance
    instrument = pm.Instrument(program=0)  # program=0 is Acoustic Grand Piano
    
    # Set up the start time and the duration for the note
    start_time = 0.0
    duration = 0.5
    
    # Add notes to the instrument
    for note_name in gen_notes:
        note_number = pm.note_name_to_number(note_name)
        note = pm.Note(
            velocity=50,   # Volume Level
            pitch=note_number, 
            start=start_time, 
            end=start_time + duration
        )
        instrument.notes.append(note)
        start_time += duration
    
    # Add the instrument to the PrettyMIDI object
    midi_data.instruments.append(instrument)
    
    # Synthesize the MIDI data directly to audio (wav)
    audio_data = midi_data.fluidsynth(fs=sample_rate)
    
    # Normalize audio to 16-bit PCM range
    audio_data = np.int16(audio_data / np.max(np.abs(audio_data)) * 32767)
    
    # Write the audio data to a .wav file
    write(file_path, sample_rate, audio_data)

In [23]:
# Set the .wav file name
wav_filename = "markov_random_notes.wav"

# Create the .wav file
create_wav_file(generated_music, wav_filename)

## Play the Generated .wav File

In [24]:
# Play the generated audio file
audio_path = wav_filename
Audio(audio_path)